In [20]:
import cv2
import math
import time
import numpy as np
import HandTrackingModule as htm

In [21]:
#global param
wCam, hCam = 640,840
pTime = 0 #initiate

#install pycaw (adjust audio)
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange() #min,max
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 0
volPer = 0

In [22]:
#Initialize some object
detector = htm.handDetector(detectionCon=0.7)



In [25]:
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

while True:
    success, img = cap.read()
    img= detector.findHands(img)

    # Find landmark list
    lmList = detector.findPosition(img, draw=False)  # Get landmark positions

    if len(lmList) != 0:
        #print("Thumb Position:", lmList[4])  # Thumb tip
        #print("Index Position:", lmList[8])  # Index tip
        #thumb tip
        x1 = lmList[4][1]
        y1 = lmList[4][2]

        #index tip
        x2 = lmList[8][1]
        y2 = lmList[8][2]

        cx,cy = (x1+x2)//2, (y1+y2)//2

        cv2.circle(img, (x1,y1),15, (255,0,255),cv2.FILLED)
        cv2.circle(img, (x2,y2),15, (255,0,255),cv2.FILLED)
        cv2.line(img, (x1,y1),(x2,y2),(0,255,255),3)
        cv2.circle(img, (cx,cy),15, (255,0,255),cv2.FILLED)

        length = math.hypot(x2-x1,y2-y1)
        #print(length) 6-210 length volrange -95 to 0
        vol = np.interp(length,[6,220],[minVol,maxVol])
        volBar = np.interp(length,[6,220],[380,10])
        volPer = np.interp(length,[6,220],[0,100])
        #print(int(length),vol)
        volume.SetMasterVolumeLevel(vol, None)



    cv2.rectangle(img, (50,10),(85,380),(0,255,0),3)
    cv2.rectangle(img, (50, int(volBar)), (85, 380), (0, 255, 0), cv2.FILLED) 
                  
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, f'FPS: {int(fps)}', (500, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
    cv2.putText(img, f'Volume: {int(volPer)}%', (40, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
    cv2.imshow("frame", img)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # Escape key
        break

cap.release()
cv2.destroyAllWindows()
